## Prepare progression data from userdata 

This notebook constructs the progression dataset for the model based on the data in the `userdata_filter.jl` file <br/>
the `userdata_filter.jl` file contains a row for each user. The first column is a dictionary containing the demographic info for that user, and the rest of the columns are lists that contain the data for each tick from the user profile (date, location, style, rating, etc). This notebook constructs a dataframe with a progression metric (`Rs`) and various summary statistics for each user's tick log. <br/>



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from scipy import stats
from random import randrange
from datetime import datetime
import seaborn as sns
import time


First load the userdata file and get rid of low tick counts in case this wasn't already done

In [5]:
#Path to the filtered userdata file
userdata_path = 'userdata_filter.json'

df = pd.read_json(userdata_path, lines=False)

tick_cutoff=100 
df = df[df.Date.str.len() > tick_cutoff]
df=df.reset_index().reset_index()

Now define a function that converts rating data to numeric (score)

In [6]:
#List of YDS grades
grades_yds = ['5.0','5.1','5.2','5.3','5.4','5.5','5.6','5.7','5.7+','5.8-','5.8','5.8+','5.9-','5.9','5.9+',
              '5.10a',['5.10-','5.10a/b'],'5.10b',['5.10','5.10b/c'],'5.10c',['5.10+','5.10c/d'],'5.10d',
              '5.11a',['5.11-','5.11a/b'],'5.11b',['5.11','5.11b/c'],'5.11c',['5.11+','5.11c/d'],'5.11d',
              '5.12a',['5.12-','5.12a/b'],'5.12b',['5.12','5.12b/c'],'5.12c',['5.12+','5.12c/d'],'5.12d',
              '5.13a',['5.13-','5.13a/b'],'5.13b',['5.13','5.13b/c'],'5.13c',['5.13+','5.13c/d'],'5.13d',
              '5.14a',['5.14-','5.14a/b'],'5.14b',['5.14','5.14b/c'],'5.14c',['5.14+','5.14c/d'],'5.14d',
              '5.15a',['5.15-','5.15a/b'],'5.15b',['5.15','5.15b/c'],'5.15c',['5.15+','5.15c/d'],'5.15d']

#Grade index - half step for slash grades, fullstep for letter grades and number grades, 1/3 step for +- below 5.10
grade_index_yds = np.array([0,1,2,3,4,5,6,7,7.33,7.66,8,8.33,8.66,9,9.5,
               10, 10.5, 11, 11.5, 12, 12.5, 13,
               14, 14.5, 15, 15.5, 16, 16.5, 17,
               18, 18.5, 19, 19.5, 20, 20.5, 21,
               22, 22.5, 23, 23.5, 24, 24.5, 25,
               26, 26.5, 27, 27.5, 28, 28.5, 29,
               30, 30.5, 31, 31.5, 32, 32.5, 33])


#Repeat for Vgrades
grades_vscale = ['V-easy', 'V0-', 'V0', ['V0+', 'V1-'], 
                 'V1', ['V1+', 'V2-'], 'V2',['V2+','V3-'],
                 'V3', ['V3+', 'V4-'],'V4',['V4+','V5-'],
                 'V5',['V5+','V6-'],'V6',['V6+','V7-'],
                 'V7',['V7+','V8-'],'V8', ['V8+','V9-'],
                 'V9',['V9+','V10-'],'V10',['V10+','V11-'],
                 'V11',['V11+','V12-'],'V12', ['V12+','V13-'],
                 'V13',['V13+','V14-'],'V14',['V14+','V15-'],
                 'V15',['V15+','V16-'],'V16', ['V16+','V17-'],
                 'V17']

grade_index_vscale = np.array([9, 9.5, 10, 11,
                      12 ,13 ,14, 15,
                      16, 17, 18, 19,
                      20, 21, 22, 23,
                      24, 25, 26, 27,
                      28, 29, 30, 31,
                      32, 33, 34, 35,
                      36, 37, 38, 39,
                      40, 41, 42, 43,
                      44])


#Create a ref score that you can easily access - YDS
grades_yds_unpacked = []
ref_scores_yds = []
for i,grade in enumerate(grades_yds):
    if isinstance(grade,str):
        ref_scores_yds.append(grade_index_yds[i])
        grades_yds_unpacked.extend([grade])
    else:
        ref_scores_yds.extend(len(grade)*[grade_index_yds[i]])
        grades_yds_unpacked.extend(grade)                    
grades_yds = grades_yds_unpacked

#Create a ref score that you can easily access - Vscale
grades_vscale_unpacked = []
ref_scores_vscale = []
for i,grade in enumerate(grades_vscale):
    if isinstance(grade,str):
        ref_scores_vscale.append(grade_index_vscale[i])
        grades_vscale_unpacked.extend([grade])
    else:
        ref_scores_vscale.extend(len(grade)*[grade_index_vscale[i]])
        grades_vscale_unpacked.extend(grade)                    
grades_vscale = grades_vscale_unpacked

#Extend to make one huge array
grades_yds.extend(grades_vscale)
ref_scores_yds.extend(ref_scores_vscale)


def grade_to_score(input_grades):
    
    #Make input a list if it's a string
    if isinstance(input_grades, str):
        input_grades=[input_grades]
        
    #Convert VX-Y grades to VX+
    input_grades = [str(grade.split('-')[0]+'+') if bool(re.match(r"(V\d+-\d+)",grade)) 
                    else grade for grade in input_grades]

            
    #Convert grades that are not in the list to 5.0 (scrambles)
    input_grades = ['5.0' if x not in grades_yds else x for x in input_grades]
                  
    output_scores = [ref_scores_yds[grades_yds.index(x)] for x in input_grades] 
    return output_scores



Define a function that converts a row of the `userdata_filter.jl` file to a dataframe containing information about each tick on the profile (excludes demographic info)

In [16]:
def produce_ticks_filterdates(df, ind, mindate, maxdate):
    #Removes ticks with dates out of the range defined by mindate and max date
    #Takes as arg the entire user dataframe and the index of the user
    #outputs df_user with date converted to datetime.

    i=ind
    #Much faster method of doing things!
    df_user = pd.DataFrame.from_dict({'Date': pd.to_datetime(df.Date[i], errors='coerce', format='%b %d, %Y'),
                                      'RatingYDS': pd.Series(df.RatingYDS[i]).tolist(),
                                      'RatingVscale': pd.Series(df.RatingVscale[i]).tolist(),
                                      'RatingMisc': pd.Series(df.RatingMisc[i]).tolist(),
                                      'RouteType': pd.Series(df.RouteType[i]).tolist(),
                                      'Location': pd.Series(df.Location[i]).tolist(),
                                      'Style': pd.Series(df.Style[i]).tolist(),
                                      'Pitches': pd.Series(df.Pitches[i]).tolist()})
    df_user = df_user.set_index('Date')
    df_user=df_user[df_user.index.to_series().between(mindate, maxdate)]
    df_user.reset_index(inplace=True) #convert date index back to a normal column....
    return df_user


Define a function that calculates the Spearmann correlation coefficient for climb ratings vs time as a measure of if the user is progressing in climb difficulty. <br/>

First we combine YDS and boulder ratings and convert that column to scores. <br/>
Then, since people can climb easy stuff on a given day (for fun, as a rest day, alpine climbing, etc), we take the highest graded route done in each month as a data point. <br/>
Then, we calculate the correlation coefficient with time 

In [18]:
#This function calculates the spearman coefficient for a user's tick df
def get_user_tickcorrelation(df_user, ploton, account_age): ## Given the tick dataframe for a single user
    #inputs is a dict with keys:factor for outlier removal and Ntake for taking highest ticks for each day
  
    #Filtering 
    df_user = df_user[df_user.Date.isna()==False] #Remove rows where date is NaN (out of bounds)
    df_user.RatingYDS.fillna(df_user.RatingVscale, inplace=True) #Fill empty YDS ratings with boulder ratings
    df_user.drop(columns=['RatingVscale'], inplace = True) #drop vscale column
    df_user.rename(columns={'RatingYDS':'Rating'},inplace=True) #rename ratingyds column
    df_user = df_user[df_user.Rating.isna()==False] #remove rows where there's no YDS rating
    scores = grade_to_score(df_user.Rating) #Calculate scores for the climbs
    df_user.drop('Rating',axis=1,inplace=True) #get rid of the rating column
    df_user['Rating']=scores #replace rating column with scores
    df_user = df_user[df_user.Rating > 0] #remove zero score values (scrambles/ice/missing values, etc)
    df_user.index=df_user.Date #set the date to be the index- necessary for grouping operation

    
    # Take the highest rated climb from each unique month in the user's ticks
    grouped = df_user.groupby(by=[df_user.index.year, df_user.index.month]) # Make date groups
    df_usercorr = pd.DataFrame.from_dict({'Rating':[]})
    df_usercorr.Rating = grouped.pipe(lambda x: x.Rating.max()) #Returns highest score in each group
    dates = [str(x[1])+'/'+str(x[0]) for x in df_usercorr.index] #date list (mont/year)
    df_usercorr['Date'] = pd.to_datetime(dates, errors='coerce', format='%m/%Y')
    
    #Calculate spearman coefficient
    x=pd.to_numeric(df_usercorr.Date).values*0.0000000000000001 #for plotting...
    y=df_usercorr.Rating.values
    rs = stats.spearmanr(x, y).correlation
    
    #Calculate number of months and fraction of months where there's a tick
    nmonths = len(x)
    tickage=((df_user.Date.iloc[1]-df_user.Date.iloc[-1]).days)/365 #tick age in years 
    frac_months_tickage = len(df_usercorr.Rating)/(12*tickage)

    if ploton:
        x= x.reshape(-1, 1)
        reg = LinearRegression().fit(x, y)
        print('Spearman coeff = '+str(rs))
        sns.regplot(x=x,y=y)

    return rs, nmonths, frac_months_tickage

Define a function to get demographics from a row of `userdata_filter.jl`

In [9]:
def get_user_demographics(UserInfo):
    #input: UserInfo (== df.UserInfo[i])
    
    if len(UserInfo['Gender'])==2: #Conversion failed in scraper
        gender=None
    else:
        gender=UserInfo['Gender']
    age=UserInfo['Age']
    accountage= ((pd.Timestamp.now()-pd.to_datetime(UserInfo['SignupDate'])).days)/365  #account age in years
    
    #Replace fake ages with None
    if age:
        if age<14 or age>80:
            age=None
    
    demovalues= {
        'Age':age,
        'Gender':gender,
        'AccountAge':accountage
    }
    
    return demovalues

Define a function to get a bunch of features from a user's tick log. These are: <br/>

`NumberTicks`: Total number of ticks<br/>
`FractionTrad`: Fraction of ticks that are trad routes<br/>
`FractionSport`: Fraction of ticks that are sport routes<br/>
`FractionBoulder`: Fraction of ticks that are boulders<br/>
`FractionDanger`: Fraction of ticks that have a danger rating (PG13, R)<br/>
`VarType`: Standard deviation in an integer encoding of route type<br/>
`VarLoc`: Number of unique areas ticks cover<br/>
`FractionStyleMarked`: Fraction of ticks where a style is marked<br/>
`FractionOnsight`: Fraction of ticks where the style is onsight<br/>
`FractionRedpoint`: Fraction of ticks where the style is redpoint<br/>
`FractionFell`: Fraction of ticks where the style is Lead/ Fell/Hung<br/>
`FractionSolo`: Fraction of ticks where the style is Solo<br/>
`FractionTR`: Fraction of ticks where the style is TR<br/>
`FractionDaysClimbing`: Fraction of all days in ticklog range that there are ticks<br/>

                 

In [24]:
def get_user_features(df_user):
#     INPUT: User tick dataframe. 
#     OUTPUT: dictionary with desired values.

    #Calculate ticklog_age
    date_series = df_user.Date[df_user.Date.isna()==False] #remove NaT
    
    #Number of days ticking
    ndays = df_user.Date.nunique()

    #Calculate the fraction of things
    nticks = len(df_user.index)
    ntrad = pd.Series((df_user.RouteType.str.contains('Trad')) & (df_user.RouteType.str.contains('Boulder')==False)).sum()/nticks
    nsport = pd.Series((df_user.RouteType.str.contains('Sport')) & (df_user.RouteType.str.contains('Boulder')==False)).sum()/nticks
    nboulder = pd.Series(df_user.RouteType.str.contains('Boulder')).sum()/nticks
    ndanger = pd.Series(df_user.RatingMisc.str.contains('R')|df_user.RatingMisc.str.contains('PG13')).sum()/nticks
    
    #Diversity in climb types - replace values with number to indicate type
    mask_trad = (df_user.RouteType.astype(str).str.contains('Trad')) & (df_user.RouteType.astype(str).str.contains('Boulder')==False)
    df_user.loc[mask_trad, 'RouteType'] = 1
    mask_sport = (df_user.RouteType.astype(str).str.contains('Sport')) & (df_user.RouteType.astype(str).str.contains('Boulder')==False)
    df_user.loc[mask_sport, 'RouteType'] = 2
    mask_boulder = (df_user.RouteType.astype(str).str.contains('Boulder')==True)
    df_user.loc[mask_boulder, 'RouteType'] = 3
    mask_misc=(df_user.RouteType.astype(str).str.isdigit())==False
    df_user.loc[mask_misc, 'RouteType'] = 4
    VarType = df_user.RouteType.std()
    
    #Diversity in locations (#of different unique locations - at the sub state level)
    VarLoc = df_user.Location.str.split(',').apply(lambda x: x[0] if x else 0).nunique()
    
    #Fraction of ticks that have a style marked 
    FractionStyleMarked = sum(df_user.Style.isna()==False)/len(df_user.index) #fraction of style that are filled
    Nmarked = sum(df_user.Style.isna()==False)
    styles = df_user.Style[df_user.Style.isna()==False]
    if Nmarked > 0:
        FractionOnsight = (sum(styles.str.contains('Onsight'))+sum(styles.str.contains('Flash')))/Nmarked
        FractionRedpoint = (sum(styles.str.contains('Send'))+sum(styles.str.contains('Redpoint')))/Nmarked
        FractionFell = (sum(styles.str.contains('Fell'))+sum(styles.str.contains('Attempt')))/Nmarked
        FractionSolo=(sum(styles.str.contains('Solo')))/Nmarked
        FractionTR = (sum(styles.str.contains('TR'))+sum(styles.str.contains('Follow')))/Nmarked
    else:
        FractionOnsight=None
        FractionRedpoint=None
        FractionFell=None
        FractionSolo=None
        FractionTR=None

    #Fraction of days spent climbing
    FractionDaysClimbing = df_user.Date.nunique()/(max(df_user.Date)-min(df_user.Date)).days
    

    
    output_dict={'NumberTicks':nticks,
                 'FractionTrad':ntrad,
                 'FractionSport':nsport,
                 'FractionBoulder':nboulder,
                 'FractionDanger':ndanger,
                 'VarType': VarType,
                 'VarLoc': VarLoc,
                 'FractionStyleMarked':FractionStyleMarked,
                 'FractionOnsight':FractionOnsight,
                 'FractionRedpoint':FractionRedpoint,
                 'FractionFell':FractionFell,
                 'FractionSolo': FractionSolo,
                 'FractionTR': FractionTR,
                 'FractionDaysClimbing': FractionDaysClimbing
                }
    return output_dict

## Build the dataframe 

In [28]:
#Initialize 

#Date range to filter ticks
mindate = '2000-01-01'
maxdate = '2023-01-01' 

tick_cutoff=50 #minimum number of ticks
days_cutoff=15 #minimum number of unique days

#Initialize empty dataframe
initial = len(df.index)*[0]
userdata = pd.DataFrame.from_dict({
    'Rs': initial,
    'Age': initial,
    'Gender': initial,
    'AccountAge':initial,
    'NumberTicks':initial,
    'Nmonths': initial,
    'FractionMonths':initial,
    'FractionTrad':initial,
    'FractionSport':initial,
    'FractionBoulder':initial,
    'FractionDanger':initial,
    'VarType': initial,
    'VarLoc': initial,
    'FractionStyleMarked':initial,
    'FractionOnsight':initial,
    'FractionRedpoint':initial,
    'FractionFell':initial,
    'FractionSolo': initial,
    'FractionTR': initial,
    'FractionDaysClimbing': initial
})


In [29]:
Rs

-0.045938977028114274

In [30]:
#######################################
#Build the dataframe iteratively
#######################################

for i in range(len(df.index)): #for each user
    
    lb = time.time()
    #Get user ticks and filter out of bounds dates
    df_user = produce_ticks_filterdates(df, i, mindate, maxdate)
    
    #Calculate the number of unique days tickin
    
    #Add to main dataframe ~if~ there are > tick_cutoff ticks remaining (after date filtering ...)
    if df_user.Date.nunique() >= days_cutoff:
        
        #Get demographic features as dictionary
        demo_features = get_user_demographics(df.UserInfo[i])

        #Calculate corr coeff.
        Rs,Nmonths,FractionMonths  = get_user_tickcorrelation(df_user, False, demo_features['AccountAge']) #false for not plotting

        #Get all features as a dictionary
        features = get_user_features(df_user)

    else: #Add none for entire row to avoid issues.filter these out later
        Rs =  None
        demo_features ={'Age':None,
                        'Gender':None,
                        'AccountAge':None
        }
        features={'Nmonths':None,
                 'NumberTicks':None,
                 'NumberDays':None,
                 'FractionMonths':None,
                 'FractionTrad':None,
                 'FractionSport':None,
                 'FractionBoulder':None,
                 'FractionAlpine':None,
                 'FractionMultipitch':None,
                 'FractionDanger':None,
                 'VarType': None,
                 'VarLoc': None,
                 'FractionStyleMarked':None,
                 'FractionOnsight':None,
                 'FractionRedpoint':None,
                 'FractionFell':None,
                 'FractionSolo': None,
                 'FractionTR': None,
                 'FractionDaysClimbing': None
                }
        
        
    #Assign values
    userdata.loc[i,'Rs']=Rs
    userdata.loc[i,'Age']=demo_features['Age']
    userdata.loc[i,'Gender']=demo_features['Gender']
    userdata.loc[i,'Nmonths']=Nmonths
    userdata.loc[i,'AccountAge']=demo_features['AccountAge']
    userdata.loc[i,'NumberTicks']=features['NumberTicks']
    userdata.loc[i,'FractionMonths']=FractionMonths
    userdata.loc[i,'FractionTrad']=features['FractionTrad']
    userdata.loc[i,'FractionSport']=features['FractionSport']
    userdata.loc[i,'FractionBoulder']=features['FractionBoulder']
    userdata.loc[i,'FractionDanger']=features['FractionDanger']
    userdata.loc[i,'VarType']=features['VarType']
    userdata.loc[i,'VarLoc']=features['VarLoc']
    userdata.loc[i,'FractionStyleMarked']=features['FractionStyleMarked']
    userdata.loc[i,'FractionOnsight']=features['FractionOnsight']
    userdata.loc[i,'FractionRedpoint']=features['FractionRedpoint']
    userdata.loc[i,'FractionFell']=features['FractionFell']
    userdata.loc[i,'FractionSolo']=features['FractionSolo']
    userdata.loc[i,'FractionTR']=features['FractionTR']
    userdata.loc[i,'FractionDaysClimbing']=features['FractionDaysClimbing']
    
    #Output time and index value
    ub = time.time()
    print('i='+str(i)+', Time to convert = '+str(ub-lb))
    
    

i=0, Time to convert = 0.0364377498626709
i=1, Time to convert = 0.022911787033081055
i=2, Time to convert = 0.023935794830322266
i=3, Time to convert = 0.021941423416137695
i=4, Time to convert = 0.023936033248901367
i=5, Time to convert = 0.025930404663085938
i=6, Time to convert = 0.022938966751098633
i=7, Time to convert = 0.0249330997467041
i=8, Time to convert = 0.027925491333007812
i=9, Time to convert = 0.026928186416625977
i=10, Time to convert = 0.028922557830810547
i=11, Time to convert = 0.02593207359313965
i=12, Time to convert = 0.023962974548339844
i=13, Time to convert = 0.024905920028686523
i=14, Time to convert = 0.02496051788330078
i=15, Time to convert = 0.02390766143798828
i=16, Time to convert = 0.024933576583862305
i=17, Time to convert = 0.028922319412231445
i=18, Time to convert = 0.0359346866607666
i=19, Time to convert = 0.023414134979248047
i=20, Time to convert = 0.03091740608215332
i=21, Time to convert = 0.024933815002441406
i=22, Time to convert = 0.0289

i=189, Time to convert = 0.024933576583862305
i=190, Time to convert = 0.03091716766357422
i=191, Time to convert = 0.03390932083129883
i=192, Time to convert = 0.023936033248901367
i=193, Time to convert = 0.02293872833251953
i=194, Time to convert = 0.0249330997467041
i=195, Time to convert = 0.021941423416137695
i=196, Time to convert = 0.03789877891540527
i=197, Time to convert = 0.025930404663085938
i=198, Time to convert = 0.038408756256103516
i=199, Time to convert = 0.023936033248901367
i=200, Time to convert = 0.024933338165283203
i=201, Time to convert = 0.03840374946594238
i=202, Time to convert = 0.04587721824645996
i=203, Time to convert = 0.0359039306640625
i=204, Time to convert = 0.0443880558013916
i=205, Time to convert = 0.024933576583862305
i=206, Time to convert = 0.023935794830322266
i=207, Time to convert = 0.02444148063659668
i=208, Time to convert = 0.04288530349731445
i=209, Time to convert = 0.02593088150024414
i=210, Time to convert = 0.024933338165283203
i=2

i=371, Time to convert = 0.04185819625854492
i=372, Time to convert = 0.02544856071472168
i=373, Time to convert = 0.0219724178314209
i=374, Time to convert = 0.028922080993652344
i=375, Time to convert = 0.023936986923217773
i=376, Time to convert = 0.027962923049926758
i=377, Time to convert = 0.024930953979492188
i=378, Time to convert = 0.021942615509033203
i=379, Time to convert = 0.02691960334777832
i=380, Time to convert = 0.02394390106201172
i=381, Time to convert = 0.021910667419433594
i=382, Time to convert = 0.02396702766418457
i=383, Time to convert = 0.022907495498657227
i=384, Time to convert = 0.032912254333496094
i=385, Time to convert = 0.025930404663085938
i=386, Time to convert = 0.023966073989868164
i=387, Time to convert = 0.026926040649414062
i=388, Time to convert = 0.03733515739440918
i=389, Time to convert = 0.021945953369140625
i=390, Time to convert = 0.03188347816467285
i=391, Time to convert = 0.024492263793945312
i=392, Time to convert = 0.0229396820068359

i=554, Time to convert = 0.02992105484008789
i=555, Time to convert = 0.02393507957458496
i=556, Time to convert = 0.024933338165283203
i=557, Time to convert = 0.021449565887451172
i=558, Time to convert = 0.026929855346679688
i=559, Time to convert = 0.022936582565307617
i=560, Time to convert = 0.023936033248901367
i=561, Time to convert = 0.021941184997558594
i=562, Time to convert = 0.024933338165283203
i=563, Time to convert = 0.02792525291442871
i=564, Time to convert = 0.02892303466796875
i=565, Time to convert = 0.022938251495361328
i=566, Time to convert = 0.023936033248901367
i=567, Time to convert = 0.021941661834716797
i=568, Time to convert = 0.023935794830322266
i=569, Time to convert = 0.02593088150024414
i=570, Time to convert = 0.02293848991394043
i=571, Time to convert = 0.025930404663085938
i=572, Time to convert = 0.026928186416625977
i=573, Time to convert = 0.0249330997467041
i=574, Time to convert = 0.024933576583862305
i=575, Time to convert = 0.028922319412231

i=739, Time to convert = 0.03490447998046875
i=740, Time to convert = 0.03889751434326172
i=741, Time to convert = 0.0229339599609375
i=742, Time to convert = 0.02194046974182129
i=743, Time to convert = 0.02792525291442871
i=744, Time to convert = 0.02393198013305664
i=745, Time to convert = 0.02294301986694336
i=746, Time to convert = 0.030918359756469727
i=747, Time to convert = 0.02592778205871582
i=748, Time to convert = 0.03845953941345215
i=749, Time to convert = 0.028896093368530273
i=750, Time to convert = 0.02396678924560547
i=751, Time to convert = 0.0269622802734375
i=752, Time to convert = 0.022938251495361328
i=753, Time to convert = 0.03390765190124512
i=754, Time to convert = 0.024929523468017578
i=755, Time to convert = 0.023971080780029297
i=756, Time to convert = 0.025931358337402344
i=757, Time to convert = 0.024936914443969727
i=758, Time to convert = 0.028923511505126953
i=759, Time to convert = 0.023931503295898438
i=760, Time to convert = 0.025934934616088867
i=

...

i=15415, Time to convert = 0.025930404663085938
i=15416, Time to convert = 0.02293848991394043
i=15417, Time to convert = 0.021941661834716797
i=15418, Time to convert = 0.0224456787109375
i=15419, Time to convert = 0.022939205169677734
i=15420, Time to convert = 0.007978200912475586
i=15421, Time to convert = 0.024933576583862305
i=15422, Time to convert = 0.022939443588256836
i=15423, Time to convert = 0.022937774658203125
i=15424, Time to convert = 0.024933576583862305
i=15425, Time to convert = 0.022938251495361328
i=15426, Time to convert = 0.021941423416137695
i=15427, Time to convert = 0.02393651008605957
i=15428, Time to convert = 0.023935556411743164
i=15429, Time to convert = 0.021941184997558594
i=15430, Time to convert = 0.02293872833251953
i=15431, Time to convert = 0.02892279624938965
i=15432, Time to convert = 0.023935794830322266
i=15433, Time to convert = 0.0219419002532959
i=15434, Time to convert = 0.0249330997467041
i=15435, Time to convert = 0.0249330997467041
i=15

i=15595, Time to convert = 0.02393794059753418
i=15596, Time to convert = 0.022936105728149414
i=15597, Time to convert = 0.024932861328125
i=15598, Time to convert = 0.025289058685302734
i=15599, Time to convert = 0.023939847946166992
i=15600, Time to convert = 0.02293848991394043
i=15601, Time to convert = 0.021941423416137695
i=15602, Time to convert = 0.02353215217590332
i=15603, Time to convert = 0.023936033248901367
i=15604, Time to convert = 0.022934436798095703
i=15605, Time to convert = 0.02194833755493164
i=15606, Time to convert = 0.022936582565307617
i=15607, Time to convert = 0.022447824478149414
i=15608, Time to convert = 0.022940874099731445
i=15609, Time to convert = 0.02492690086364746
i=15610, Time to convert = 0.021945714950561523
i=15611, Time to convert = 0.021943092346191406
i=15612, Time to convert = 0.023934125900268555
i=15613, Time to convert = 0.02393198013305664
i=15614, Time to convert = 0.022942543029785156
i=15615, Time to convert = 0.024935483932495117
i

i=15774, Time to convert = 0.02290797233581543
i=15775, Time to convert = 0.021972179412841797
i=15776, Time to convert = 0.02194666862487793
i=15777, Time to convert = 0.022933483123779297
i=15778, Time to convert = 0.021931886672973633
i=15779, Time to convert = 0.021942138671875
i=15780, Time to convert = 0.022937536239624023
i=15781, Time to convert = 0.02593255043029785
i=15782, Time to convert = 0.006949186325073242
i=15783, Time to convert = 0.022972822189331055
i=15784, Time to convert = 0.022914409637451172
i=15785, Time to convert = 0.020965099334716797
i=15786, Time to convert = 0.006949901580810547
i=15787, Time to convert = 0.021968364715576172
i=15788, Time to convert = 0.022942304611206055
i=15789, Time to convert = 0.007976770401000977
i=15790, Time to convert = 0.02194356918334961
i=15791, Time to convert = 0.021941184997558594
i=15792, Time to convert = 0.022938966751098633
i=15793, Time to convert = 0.021938323974609375
i=15794, Time to convert = 0.006953954696655273

i=15949, Time to convert = 0.023935794830322266
i=15950, Time to convert = 0.021941423416137695
i=15951, Time to convert = 0.0249330997467041
i=15952, Time to convert = 0.021941661834716797
i=15953, Time to convert = 0.02294182777404785
i=15954, Time to convert = 0.021938800811767578
i=15955, Time to convert = 0.02293848991394043
i=15956, Time to convert = 0.02293872833251953
i=15957, Time to convert = 0.024933338165283203
i=15958, Time to convert = 0.02593064308166504
i=15959, Time to convert = 0.021941423416137695
i=15960, Time to convert = 0.02293848991394043
i=15961, Time to convert = 0.02293872833251953
i=15962, Time to convert = 0.02246880531311035
i=15963, Time to convert = 0.022929668426513672
i=15964, Time to convert = 0.02293872833251953
i=15965, Time to convert = 0.021941184997558594
i=15966, Time to convert = 0.022938966751098633
i=15967, Time to convert = 0.023935556411743164
i=15968, Time to convert = 0.024933338165283203
i=15969, Time to convert = 0.026928186416625977
i=

i=16125, Time to convert = 0.022938966751098633
i=16126, Time to convert = 0.023935556411743164
i=16127, Time to convert = 0.036901235580444336
i=16128, Time to convert = 0.02293872833251953
i=16129, Time to convert = 0.02293848991394043
i=16130, Time to convert = 0.02692866325378418
i=16131, Time to convert = 0.021940946578979492
i=16132, Time to convert = 0.021941184997558594
i=16133, Time to convert = 0.024933338165283203
i=16134, Time to convert = 0.02393627166748047
i=16135, Time to convert = 0.023935317993164062
i=16136, Time to convert = 0.021941661834716797
i=16137, Time to convert = 0.02293848991394043
i=16138, Time to convert = 0.020975589752197266
i=16139, Time to convert = 0.00797891616821289
i=16140, Time to convert = 0.021465063095092773
i=16141, Time to convert = 0.021941184997558594
i=16142, Time to convert = 0.024933815002441406
i=16143, Time to convert = 0.021974563598632812
i=16144, Time to convert = 0.020942211151123047
i=16145, Time to convert = 0.0219419002532959


In [35]:
# #Save to a file after removing NaN rows:

userdata = userdata[userdata.Rs.isna()==False]

filepath = 'userdata_formatted.json' #Path to file
userdata.to_json(filepath)
